# Constants


In [1]:

import pandas as pd
import os
import shutil
from os import path as osp

import torch
from tiatoolbox.utils.misc import select_device
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
import json
import glob

# from src.intensity import add_features_and_create_new_dicts

from src.featureextraction import get_cell_features, add_features_and_create_new_dicts
from src.train import stratified_split, recur_find_ext, run_once, rm_n_mkdir ,reset_logging
from src.graph_construct import create_graph_with_pooled_patch_nodes, get_pids_labels_for_key


ON_GPU = False

device = select_device(on_gpu=ON_GPU)

SEED = 5
random.seed(SEED)
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
STAIN = 'BCL2'


FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
PATCH_SIZE = 224
OUTPUT_SIZE = PATCH_SIZE*8

WORKSPACE_DIR = Path(BASEDIR)
GRAPH_DIR = WORKSPACE_DIR / "graphs"
LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"


# Graph construction
# PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500


GRAPHSDIR = f'{BASEDIR}/graphs'
LABELSPATH = f'{BASEDIR}/graphs/0_labels.json'

NUM_EPOCHS = 100
NUM_NODE_FEATURES = 128
NCLASSES = 3

TRAIN_DIR = WORKSPACE_DIR / "training"
SPLIT_PATH = TRAIN_DIR / "splits.dat"
RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{datetime.now().strftime('%m_%d_%H_%M_%S')}"
RUN_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR = RUN_OUTPUT_DIR / "model"

# Extract features: 20 minutes

In [10]:
   
##########
# read annotation csv, filter, and process intensity features
df = pd.read_csv(ANNPATH)

df = df[df['stain'] == STAIN]
df['area'] = (df['xe'] - df['xs']) *  (df['ye'] - df['ys'])/10000
df = df[df['area'] >= 150]  
df = df[df['xs']  >=0 ]
df = df[df['ys']  >=0 ]
df = df[df['xe']  >=0 ]
df = df[df['ye']  >=0 ]

df = df.reset_index()
##########

###############
# add intensity features
start_index = 0
end_index = len(df.index)

datpaths = []
imgpaths = []
updatpaths = []

# for index in range(start_index, 1):
for index in range(start_index, end_index):
    df_index = df['index'][index]
    patient_id = df['patient_id'][index]
    stain = df['stain'][index]
    tma_id = df['tma_id'][index]
    unique_id = str(patient_id) + '_' + stain + '_' + str(df_index)

    img_file_name = f"{FIDIR}/images/{patient_id}/{patient_id}_{stain}_{tma_id}_{OUTPUT_SIZE}_{df_index}.png"
    dat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/0.dat"
    updat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/{unique_id}.dat"

    datpaths.append(dat_file_name)
    imgpaths.append(img_file_name)
    updatpaths.append(updat_file_name)
    


In [11]:
FEATSCALERPATH

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/BCL2/0_feat_scaler.npz'

In [12]:
# 20 minutes
# shutil.rmtree(FEATSDIR)
# if not osp.exists(FEATSDIR):
#     os.makedirs(FEATSDIR)
add_features_and_create_new_dicts(datpaths, imgpaths, updatpaths)

  5%|▍         | 18/385 [00:54<18:40,  3.05s/it]


KeyboardInterrupt: 

In [ ]:
# 4 minutes
gns = StandardScaler()
for featpath in tqdm(updatpaths):
    # print(featpath)
    celldatadict = joblib.load(featpath)
    cellsfeats = np.array([v['intensity_feats'] for k, v in celldatadict.items()])
    gns.partial_fit(cellsfeats)

np.savez(FEATSCALERPATH, mean=gns.mean_, var=gns.var_)

# dd = np.load(FEATSCALERPATH)
# print(dd['mean'], gns.mean_)
# print(dd['var'], gns.var_)

# Prepare graphs : 4 minutes

In [2]:

# Graph construction
PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500


# divide into quantiles
df = pd.read_csv(CLINPATH)
df_labels = get_pids_labels_for_key(df, key='OS', nclasses=3)

# # save paths
# featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
# featpaths = [x if "/0.dat" not in x for x in featpaths]
featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
featpaths = [x for x in featpaths if ("/0.dat" not in x) and ("/file_map.dat" not in x)]
featpaths
pids = [int(osp.basename(featpath).split('_')[0]) for featpath in featpaths]
df_featpaths = pd.DataFrame(zip(pids, featpaths), columns=['patient_id', 'featpath'])

# merge to find datapoints with graph data and labels
df_data = df_featpaths.merge(df_labels, on='patient_id')
# df_data = df_data[:12]

featpaths_data = df_data['featpath'].to_list()
labels_data = df_data['OS_class'].to_list()
outgraphpaths_data = [f"{GRAPHSDIR}/{osp.basename(featpath).split('.')[0]}.json" for featpath in featpaths_data]

# save labels
labels_dict = {osp.basename(graphpath): label for graphpath, label in zip(outgraphpaths_data, labels_data)}
with open(LABELSPATH, 'w') as f:
    json.dump(labels_dict, f)


# read normalizer stats from file and pass to fn
dd = np.load(FEATSCALERPATH)
cell_feat_norm_stats = (dd['mean'], dd['var'])

# create final graphs data
# shutil.rmtree(GRAPHSDIR)
# if not osp.exists(GRAPHSDIR):
    # os.makedirs(GRAPHSDIR)
create_graph_with_pooled_patch_nodes(
    featpaths_data,
    labels_data,
    outgraphpaths_data,
    PATCH_SIZE,
    cell_feat_norm_stats=cell_feat_norm_stats,
    MIN_CELLS_PER_PATCH= MIN_CELLS_PER_PATCH,
    CONNECTIVITY_DISTANCE = CONNECTIVITY_DISTANCE
)
    # wont work in parallel mode
    # print(np.mean(global_patch_stats), np.std(global_patch_stats))

  0%|          | 0/319 [00:00<?, ?it/s]|2023-11-20|23:57:30.896| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()

|2023-11-20|23:57:30.897| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-usin

# Train model

In [3]:
wsi_paths = recur_find_ext(GRAPHSDIR, [".json"])
wsi_names = [Path(v).stem for v in wsi_paths]
assert len(wsi_paths) > 0, "No files found."  # noqa: S101

len(wsi_paths) , len(wsi_names)

(320, 320)

In [4]:
NUM_FOLDS = 1
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8 * 0.7
VALID_RATIO = 0.8 * 0.3

if SPLIT_PATH and os.path.exists(SPLIT_PATH):
    splits = joblib.load(SPLIT_PATH)
else:
    x = np.array(wsi_names)
    with open(LABELSPATH, 'r') as f:
        labels_dict = json.load(f)
    print(labels_dict)
    y = np.array([labels_dict[wsi_name+'.json'] for wsi_name in wsi_names])
    splits = stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)
    joblib.dump(splits, SPLIT_PATH)


In [5]:
# # we must define the function after training/loading
# def nodes_preproc_func(node_features: np.ndarray) -> np.ndarray:
#     """Pre-processing function for nodes."""
#     return node_scaler.transform(node_features)
nodes_preproc_func = None


splits = joblib.load(SPLIT_PATH)
loader_kwargs = {
    "num_workers": 6,
    "batch_size": 32,
}
# arch_kwargs = {
#     "dim_features": NUM_NODE_FEATURES,
#     "dim_target": NCLASSES,
#     "layers": [32, 32, 16, 8],
#     "dropout": 0.3,
#     "pooling": "mean",
#     "conv": "EdgeConv",
#     "aggr": "max",
# }


arch_kwargs = {
    "dim_features": NUM_NODE_FEATURES,
    "dim_target": NCLASSES,
    "layers": [64, 32, 8],
    "dropout": 0.3,
    "pooling": "mean",
    "conv": "EdgeConv",
    "aggr": "max",
}
optim_kwargs = {
    "lr": 5.0e-4,
    "weight_decay": 1.0e-4,
}

NUM_EPOCHS = 100
# if not MODEL_DIR.exists() or True:
for split_idx, split in enumerate(splits):
        new_split = {
            "train": split["train"],
            "infer-train": split["train"],
            "infer-valid-A": split["valid"],
            "infer-valid-B": split["test"],
        }
        MODEL_DIR = Path(MODEL_DIR) 
        split_save_dir = MODEL_DIR / f"{split_idx:02d}/"
        rm_n_mkdir(split_save_dir)
        reset_logging(split_save_dir)
        run_once(
            new_split,
            NUM_EPOCHS,
            save_dir=split_save_dir,
            arch_kwargs=arch_kwargs,
            loader_kwargs=loader_kwargs,
            optim_kwargs=optim_kwargs,
            on_gpu=ON_GPU,
            GRAPH_DIR=GRAPH_DIR
        )

100%|██████████| 7/7 [00:00<00:00, 15.54it/s]
|2023-11-20|23:59:17.870| [INFO] train-EMA-loss: 0.914402
|2023-11-20|23:59:18.214| [INFO] infer-train-accuracy: 0.454148
|2023-11-20|23:59:18.415| [INFO] infer-valid-A-accuracy: 0.307692
|2023-11-20|23:59:18.651| [INFO] infer-valid-B-accuracy: 0.406250
|2023-11-20|23:59:18.909| [INFO] EPOCH: 017


best_score {'infer-train-accuracy': 0.4978165938864629, 'infer-valid-A-accuracy': 0.46153846153846156, 'infer-valid-B-accuracy': 0.421875}


100%|██████████| 7/7 [00:00<00:00, 14.91it/s]
|2023-11-20|23:59:19.381| [INFO] train-EMA-loss: 0.974113
|2023-11-20|23:59:19.759| [INFO] infer-train-accuracy: 0.480349
|2023-11-20|23:59:19.977| [INFO] infer-valid-A-accuracy: 0.346154
|2023-11-20|23:59:20.203| [INFO] infer-valid-B-accuracy: 0.390625
|2023-11-20|23:59:20.445| [INFO] EPOCH: 018


best_score {'infer-train-accuracy': 0.4978165938864629, 'infer-valid-A-accuracy': 0.46153846153846156, 'infer-valid-B-accuracy': 0.421875}


100%|██████████| 7/7 [00:00<00:00, 15.24it/s]
|2023-11-20|23:59:20.907| [INFO] train-EMA-loss: 0.853936
|2023-11-20|23:59:21.234| [INFO] infer-train-accuracy: 0.458515
|2023-11-20|23:59:21.483| [INFO] infer-valid-A-accuracy: 0.423077
|2023-11-20|23:59:21.744| [INFO] infer-valid-B-accuracy: 0.406250
|2023-11-20|23:59:22.009| [INFO] EPOCH: 019


...

best_score {'infer-train-accuracy': 0.7292576419213974, 'infer-valid-A-accuracy': 0.46153846153846156, 'infer-valid-B-accuracy': 0.5}


100%|██████████| 7/7 [00:00<00:00, 13.25it/s]
|2023-11-21|00:00:02.797| [INFO] train-EMA-loss: 0.635997
|2023-11-21|00:00:03.169| [INFO] infer-train-accuracy: 0.716157
|2023-11-21|00:00:03.381| [INFO] infer-valid-A-accuracy: 0.423077
|2023-11-21|00:00:03.630| [INFO] infer-valid-B-accuracy: 0.468750
|2023-11-21|00:00:03.889| [INFO] EPOCH: 046


best_score {'infer-train-accuracy': 0.7292576419213974, 'infer-valid-A-accuracy': 0.46153846153846156, 'infer-valid-B-accuracy': 0.5}


100%|██████████| 7/7 [00:00<00:00, 14.42it/s]
|2023-11-21|00:00:04.377| [INFO] train-EMA-loss: 0.658772
|2023-11-21|00:00:04.726| [INFO] infer-train-accuracy: 0.742358
|2023-11-21|00:00:04.932| [INFO] infer-valid-A-accuracy: 0.461538
|2023-11-21|00:00:05.170| [INFO] infer-valid-B-accuracy: 0.453125


KeyboardInterrupt: 